In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint
import pickle

# Load the pickle files we created in our load_images_notebook file
X_f = pickle.load(open("X_features.pickle", "rb"))
y_l = pickle.load(open("y_labels.pickle", "rb"))

# Normalize the data
X_f = tf.keras.utils.normalize(X_f, axis=1)

# Construct the model
model = Sequential()

#Conv2D: Start with a convolutional layer
#3x3 window
model.add(Conv2D(64, (3,3), input_shape = X_f.shape[1:])) 
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
# Now we do it all again!
model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

# For good measure, put the data through a final dense layer at the end
# Need to flatten the data from its 2D form for convolutional to 1D form
# for the dense layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(.5))
# Final output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))
# In this instance we will use binary crossentropy since it is Clean vs. Contaminated
model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.25,
                              patience=5, min_lr=0.001)
model.fit(X_f, y_l, batch_size=32, validation_split=0.25, epochs=40, callbacks=[reduce_lr])


E:\Users\Elliott\Anaconda3\envs\TensorFlow Testing\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 68 samples, validate on 23 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
68/68 [==============================] - 4s 52ms/sample - loss: 0.7632 - acc: 0.5000 - val_loss: 0.6737 - val_acc: 0.6087
Epoch 2/40
68/68 [==============================] - 1s 8ms/sample - loss: 0.7008 - acc: 0.5735 - val_loss: 0.6581 - val_acc: 0.6087
Epoch 3/40
68/68 [==============================] - 1s 8ms/sample - loss: 0.6755 - acc: 0.6471 - val_loss: 0.6319 - val_acc: 0.6087
Epoch 4/40
68/68 [==============================] - 1s 8ms/sample - loss: 0.6097 - acc: 0.6324 - val_loss: 0.6500 - val_acc: 0.6087
Epoch 5/40
68/68 [==============================] - 1s 8ms/sample - loss: 0.5604 - acc: 0.6176 - val_loss: 0.6298 - val_acc: 0.6087
Epoch 6/40
68/68 [==============================] - 1s 9ms/s

In [2]:
# Save the model
# model.save('basic_binary_classifier.model')

In [3]:
# new_model = tf.keras.models.load_model('basic_binary_classifier.model')

In [7]:
# https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras?answertab=votes#tab-top
import tensorflow.keras.backend as K
import numpy as np
# uncertaintyFunction = K.function([model.layers[0].input, K.learning_phase()],
#                [model.layers[-1].output])

# def predict_with_uncertainty(f, x, n_iter=91):
#     result = []
#     for i in range(n_iter):
#         result.append(f([x, 1]))
    
#     result = numpy.array(result)

#     prediction = result.mean(axis=0)
#     uncertainty = result.var(axis=0)
#     return prediction, uncertainty
# prediction, uncertainty = predict_with_uncertainty(uncertaintyFunction, X_f)
# print(prediction)

In [8]:
# new_model = tf.keras.models.load_model('E:/Users/Elliott/dev/aquavision_project/basic_binary_classifier.model')
val_loss, val_acc = model.evaluate(X_f, y_l)
print(val_loss, val_acc)
np_test = np.array(X_f)
# print(np_test)
# Predict wants a numpy array as its parameter
predictions = model.predict(X_f)
# print(predictions)

91/91 [==============================] - 1s 6ms/sample - loss: 0.2426 - acc: 0.9780
0.24261264290128434 0.978022


In [9]:
import matplotlib.pyplot as plt
k = 0
print("The prediction for the image at index {} is: {}".format(k, np.rint(predictions[k])))
print("The correct value for the image at index {} is: {}".format(k, y_l[k]))

numCorr = 0
for k in range(0, len(predictions)):
    if(np.rint(predictions[k]) == y_l[k]):
        numCorr+=1
    else:
        print(k)
#         Print nums of images we didn't get
#         Indexes 68 and 78 (#69 and 79) are consistently missed in every single version of this model. 
#         Index 6 (#7) also causes problems.
print("numCorr: {}".format(numCorr))
print("Accuracy is: {}".format(numCorr/len(predictions)))

The prediction for the image at index 0 is: [1.]
The correct value for the image at index 0 is: 1
68
78
numCorr: 89
Accuracy is: 0.978021978021978
